# Chapter 14: Surrogate Models

In [1]:
import numpy as np
from itertools import product

## Algorithm 14.1

In [61]:
def design_matrix(X):
    n, m = len(X[0]), len(X)
    return np.array([ [1.0 if j == 0 else X[i][j-1] for j in range(n+1)]for i in range(m)])

def linear_regression(X,y):
    theta = np.linalg.solve(design_matrix(X),y)
    f = lambda x: np.matmul(theta,np.array([1]+x))
    return f

### Example

In [64]:
X = [[2,3,4],[3,2,4],[3,4,1],[3,4,3]]
y = [3,2,1,6]
model = linear_regression(X,y)
model([2,3,4])

[-21.25   2.25   3.25   2.5 ]


3.0

## Algorithm 14.2

In [ ]:
from itertools import product
def regression(X,y, bases):
    B =[ b(x) for x, b in product(X, bases) ]
    theta = np.linalg.solve(B, y)
    f = lambda x: np.sum([theta[i]*bases[i](x) for i in range(len(theta))])
    return f

## Algorithm 14.3

In [149]:
def polynomial_bases_1d(i,k):
    bases = []
    for p in range(k+1):
        def func(x,p=p):
            return x[i]**p
        bases.append(func)
    return bases

def polynomial_bases(n,k):
    bases = [ polynomial_bases_1d(i, k) for i in range(n)]
    terms = []
    for ks in product(*[list(np.arange(0,k+1)) for i in range(n)]):
        if np.sum(ks) <= k:
            def func(x, ks=ks):
                return np.prod([b[j](x) for j, b in zip(ks,bases)])  # Problemn with bidding arguments to this function !!
            terms.append(func)

    return terms

## Algorithm 14.4

In [ ]:
def sinusoidal_bases_1d(j, k, a, b):
    T= b[j] - a[j]
    bases = [lambda x: .5]
    for i in range(k+1):
        def func(x,i=i):
            return np.sin(2*np.pi*i*x[j]/T)
        bases.append(func)
        def func2(x,i=i):
            return np.cos(2*np.pi*i*x[j]/T)
        bases.append(func2) 
    return bases

def sinusoidal_bases(k, a, b):
    n = len(a)
    bases = [ sinusoidal_bases_1d(i, k, a, b) for i in range(n)]
    terms = []
    for ks in product(*[list(np.arange(0,(2*k)+1)) for i in range(n)]):
        powers = [np.div(k+1,2) for k in ks]
        if np.sum(powers) <= k:
            def func(x, ks=ks):
                return np.prod([b[j](x) for j, b in zip(ks,bases)])  # Problemn with bidding arguments to this function !!
            terms.append(func)

    return terms

## Algorithm 14.5

In [ ]:
def radial_bases(psi, C, p=2):
    bases = []
    for c in C:
        def func(x,c=c):
            return psi(np.linalg.norm(x-c,p))
        bases.append(func)
    return bases